# Thoughts
- I could embed the URLs or YouTube links into in embedding space outputting a vector
- I can consider each sentence individually as a classsification problem, predictind a type from each sentence and then averaging all the sentence classes
- Gonna change it so the indiviudal posts are just one long paragraph

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as layers
from tensorflow.keras import Input
from absl import logging
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import urllib
from custom_gen import DataGenerator
import glob
import json

Here we're going to load the universal sentence encoder which is a encoder created by Google that can encode any Word sentence or paragraph to a 512 dimensional vector.

In [7]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embedding_model = hub.load(module_url)
def embed(input):
  return embedding_model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
original_data = pd.read_csv("data/mbti_1.csv")
def remove_links_and_embed(posts):
    embed([re.sub("(?P<url>https?://[^\s]+)", '', post) for post in posts.split('|||')])

original_data['posts'] = original_data['posts'].apply(remove_links_and_embed)

code = {tp:i for i,tp in enumerate(pd.unique(original_data['type']))}
rev_code  = {i:tp for i,tp in enumerate(pd.unique(original_data['type']))}

In [2]:
def get_params(data):
    max_imgs= 0,
    for i in range(data.shape[0]):
        if len(imgs[i]) > max_imgs:
            max_imgs = len(imgs[i])
    return max_imgs

In this section we need to pull out the URLs from the user posts

In [ ]:
imgs = {i:[] for i in range(data.shape[0])}

for i, person in enumerate(data['posts']):
    urls_person = [re.findall("(?P<url>https?://[^\s]+)", post) for post in person.split('|||')]
    w = [re.sub("(?P<url>https?://[^\s]+)", '', post) for post in person.split('|||')]
    urls_clean = []
    for url, ws in zip(urls_person,w): 
        if url:
            urls_clean.append(url[0])
        if ws:
            words[i].append(re.sub(' +', ' ',ws))
    for url in urls_clean:
        if (".jpg" in url) or (".png" in url):
            imgs[i].append(url)

In [8]:
# encoding = np.array([list(map(float,i[1:-1].split(','))) for i in data['USE']])

In [54]:
X_train, X_test, y_train, y_test = train_test_split(np.stack(sen_data.values),np.array(data['type']),test_size=.1)

In [11]:
# Parameters
params = {'dim': (512,1),
          'batch_size': 64,
          'n_classes': 16,
          'n_channels': 1,
          'shuffle': True}

# Datasets
sentence_names = glob.glob('data/*.npy')
names_train, names_test = train_test_split(sentence_names,test_size=.1)
f = open('labels.txt', 'r')
labels = json.loads(f.read())
partition = {'train':names_train,'test':names_test}

training_generator = DataGenerator(partition['train'], labels, **params)
test_generator = DataGenerator(partition['test'], labels, **params)

# Design model
model = Sequential()
model.add(layers.Conv1D(128,5,input_shape=(512,1)))
model.add(layers.MaxPooling1D())
model.add(layers.Conv1D(64,1))
model.add(layers.MaxPooling1D())

model.add(layers.Flatten())

model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dropout(.2))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dropout(.2))
model.add(layers.Dense(16,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(generator=training_generator,
                    epochs=10,
                    use_multiprocessing=True,
                    workers=4,
                    validation_data=test_generator)

Epoch 1/10
 399/5796 [=>............................] - ETA: 3:15 - loss: 2.4299 - accuracy: 0.1779

KeyboardInterrupt: 

In [65]:
test = embed(["I eat shit for breakfast. Good thing its not vegan"])
rev_code[np.argmax(model.predict(test))]

'INTP'